# Python Pandas Review

In [ ]:
# 패키지를 호출합니다.
import numpy as np
import pandas as pd

### 외부 파일을 데이터프레임으로 불러오기

In [ ]:
df = pd.read_csv('https://bit.ly/2019_KBO_hitters')
df

In [ ]:
type(df)

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
df.dtypes

### 데이터프레임 다루기 : 인덱스로 일부 확인

In [ ]:
df.iloc[[0]]

In [ ]:
df.iloc[0:10]

In [ ]:
df.iloc[0:10, : ]

In [ ]:
df.iloc[0:10, 0:12]

In [ ]:
df.iloc[[1, 10], [0, 6, 3]]

In [ ]:
df.index = range(1, 301)
df

In [ ]:
df.loc[[1]]

In [ ]:
df.loc[1:10]

In [ ]:
df.loc[1:10, '선수명':'도루']

In [ ]:
df.loc[[1, 10], ['선수명', '홈런', '타석']]

### 데이터프레임 묶기 : 행 기준으로 합치기

In [ ]:
df1 = df.loc[1:5, '선수명':'도루']
df1

In [ ]:
df2 = df.loc[6:10, '선수명':'도루']
df2

In [ ]:
pd.concat([df1, df2], axis = 0)

### 데이터프레임 묶기 : 열 기준으로 합치기

In [ ]:
df1 = df.loc[1:10, '선수명':'안타']
df1

In [ ]:
df2 = df.loc[1:10, '홈런':'도루']
df2

In [ ]:
pd.concat([df1, df2], axis = 1)

### 데이터프레임 병합(merge)

In [ ]:
df1 = df.loc[1:22:3, '선수명':'경기']
df1

In [ ]:
df2 = df.loc[1:15:2, ['선수명', '타수', '안타']]
df2

In [ ]:
pd.merge(df1, df2)

In [ ]:
pd.merge(df1, df2, how = 'outer')

In [ ]:
pd.merge(df1, df2, how = 'left')

In [ ]:
pd.merge(df1, df2, how = 'right')

### 데이터프레임 객체 전처리 : 일부 컬럼명 변경

In [ ]:
df1.columns = ['이름', '팀명', '경기']
df1

In [ ]:
df1 = df1.rename(columns = {'이름':'선수이름'})
df1

In [ ]:
pd.merge(df1, df2, left_on = '선수이름', right_on = '선수명')

### 데이터프레임 객체 전처리 : 일부 컬럼만 선택

In [ ]:
df['팀명']

In [ ]:
df[['팀명']]

In [ ]:
df.loc[1:10, ['팀명']]

In [ ]:
df[['선수명', '팀명']]

In [ ]:
df.loc[1:10, ['선수명', '팀명']]

### 데이터프레임 객체 전처리 : 일부 컬럼만 삭제

In [ ]:
df.drop(range(101, 301))

In [ ]:
df.drop(['BABIP'], axis = 1, inplace = True)
df

In [ ]:
df.drop(['wOBA', 'WAR'], axis = 1, inplace = True)
df

### 데이터프레임 객체 전처리 : 컬럼별 타입 변환

In [ ]:
df['팀명'] = pd.Categorical(df['팀명'])
df['팀명']

In [ ]:
df['팀명'].value_counts()

In [ ]:
df = df.astype({'경기': str, '타석': float})
df.dtypes

In [ ]:
df.loc[:, '경기':'타석'] = df.loc[:, '경기':'타석'].astype(int)
df.dtypes

### 데이터프레임 객체 전처리 : 결측값 제거

In [ ]:
df.isna()

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.info()

### 데이터프레임 객체 전처리 : 컬럼 간 연산

In [ ]:
df['타수*'] = df['타석'] - df['타수']
df[['선수명', '타석', '타수', '타수*']]

In [ ]:
df['타율*'] = np.round(df['안타'] / df['타수'], 3)
df[['선수명', '타율', '타율*']]

In [ ]:
df['OPS*'] = df['출루율'] + df['장타율']
df[['선수명', 'OPS', 'OPS*']]

In [ ]:
df = df.loc[ : , '선수명':'OPS']

### 데이터프레임 객체 전처리 : 조건을 만족하는 행 선택

In [ ]:
df = df[df['타석'] >= 144 * 3.1]
df

In [ ]:
df.shape

In [ ]:
df.reset_index()

In [ ]:
df = df.reset_index(drop = True)
df

In [ ]:
df.index = range(1, 55)
df

In [ ]:
df.sort_values(by = '안타', ascending = False).head(5)

In [ ]:
df.sort_values(by = ['홈런', '삼진'], ascending = False).head(5)

In [ ]:
df.sort_values(by = ['홈런', '삼진'], ascending = [False, True]).head(5)

In [ ]:
df[(df['홈런'] >= 20) & (df['OPS'] >= 0.9)]

In [ ]:
df[(df['홈런'] >= 20) & (df['도루'] >= 20)]

## dfply 패키지 소개

In [ ]:
# !pip install dfply

In [ ]:
from dfply import *

### 특정 조건을 만족하는 행과 일부 컬럼 선택

In [ ]:
df = pd.read_csv('https://bit.ly/2019_KBO_hitters')

In [ ]:
df = (df >> 
      filter_by(X.타석 >= 1) >> 
      select(X.팀명, X.선수명, X.타수, X.안타, X.홈런))
df.shape

### 집계함수로 팀타율 데이터 생성

In [ ]:
team1 = (df >>
         group_by(X.팀명) >>
         summarize(팀타수 = X.타수.sum(),
                   팀안타 = X.안타.sum()) >>
         mutate(팀타율 = np.round(X.팀안타 / X.팀타수, 3)))

In [ ]:
team1.sort_values(by = '팀타율', ascending = False, inplace = True)
team1

### [참고] pandas 함수로 팀타율 데이터 생성

In [ ]:
팀안타 = df.groupby('팀명').sum()['안타']
팀타수 = df.groupby('팀명').sum()['타수']
팀타율 = np.round(팀안타 / 팀타수, 3)
팀타율.sort_values(ascending = False, inplace = True)
팀타율

### 집계함수로 팀별 인당 평균 홈런 데이터 생성

In [ ]:
team2 = (df >>
         group_by(X.팀명) >>
         summarize(팀홈런 = X.홈런.sum(),
                   팀인원 = X.홈런.count()) >>
         mutate(인당홈런 = np.round(X.팀홈런 / X.팀인원, 2)))

In [ ]:
team2.sort_values(by = '인당홈런', ascending = False, inplace = True)
team2

### [참고] pandas 함수로 팀별 인당 평균 홈런 데이터 생성

In [ ]:
팀홈런 = df.groupby('팀명').sum()['홈런']
팀인원 = df.groupby('팀명').count()['선수명']
인당홈런 = np.round(팀홈런 / 팀인원, 2)
인당홈런.sort_values(ascending = False, inplace = True)
인당홈런

## End of Document